In [1]:
df = sqlContext.read.csv('s3a://sparkdemonstration/10M.csv', header=True, inferSchema=True)
df.printSchema()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1589595603798_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- click: boolean (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- 

In [2]:
from pyspark.sql.types import IntegerType
from math import floor
from pyspark.sql.functions import rand
from pyspark.sql.functions import col

def stratifiedSample(df, N, labelCol="y"):
    ctx = df.groupby(labelCol).count()
    ctx = ctx.withColumn('frac', col("count") / df.count())
    frac = ctx.select("y", "frac").rdd.collectAsMap()
    pos = int(floor(frac[1] * N))
    neg = int(floor(frac[0] * N))
    posDF = df.filter(col(labelCol) == 1).orderBy(rand()).limit(pos)
    negDF = df.filter(col(labelCol) == 0).orderBy(rand()).limit(neg)
    return posDF.unionAll(negDF)

df = df.withColumn("y", df["click"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
xdf = stratifiedSample(df, 5_000_000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
categorical = ['banner_pos', 'site_category', 'site_domain', 'device_type', 'app_domain', 'app_category',
               'device_model']
numerical = ['month', 'day', 'hour']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.ml.feature import StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
xdf.select(*categorical).limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+-----------+-----------+----------+------------+------------+
|banner_pos|site_category|site_domain|device_type|app_domain|app_category|device_model|
+----------+-------------+-----------+-----------+----------+------------+------------+
|         0|     50e219e0|   948ff336|          0|  7801e8d9|    07d7df22|    66960a7d|
|         0|     28905ebd|   f3845767|          1|  7801e8d9|    07d7df22|    3bd9e8e7|
|         0|     50e219e0|   c4e18dd6|          1|  5c5a694b|    0f2161f8|    d6e0e6ff|
|         0|     28905ebd|   f3845767|          1|  7801e8d9|    07d7df22|    8ce34352|
|         0|     28905ebd|   f3845767|          1|  7801e8d9|    07d7df22|    edead9f4|
+----------+-------------+-----------+-----------+----------+------------+------------+

In [7]:
stringCols = ['site_category', 'site_domain', 'app_domain', 'app_category', 'device_model']
newCols = []
for c in stringCols:
    si = StringIndexer(inputCol=c, outputCol=c + '_ix')
    xdf = si.fit(xdf).transform(xdf)
    newCols.append(c + '_ix')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
cCols = newCols + ['banner_pos', 'device_type']
cCols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['site_category_ix', 'site_domain_ix', 'app_domain_ix', 'app_category_ix', 'device_model_ix', 'banner_pos', 'device_type']

In [9]:
from pyspark.ml.feature import OneHotEncoderEstimator

ohe = OneHotEncoderEstimator(inputCols=cCols, outputCols=[c+'Enc' for c in cCols])
oh_encoder = ohe.fit(xdf)
xdf = oh_encoder.transform(xdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
trainCols = [c+'Enc' for c in cCols] + numerical
assembler = VectorAssembler(inputCols=trainCols, outputCol='features')
xdf = assembler.transform(xdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
lr = LogisticRegression(featuresCol='features', labelCol='y')
model = lr.fit(xdf)
result = model.evaluate(xdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
result.accuracy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8324411664882333

In [15]:
result.areaUnderROC

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7262387526956587

In [16]:
result.recallByLabel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.9902967281631213, 0.060671732852155644]